# ¡Ha habido un asesinato en SQL City! 

*SQL Murder Mystery está diseñado para ser tanto una lección autodirigida para aprender conceptos y comandos de SQL como un juego divertido para que los usuarios experimentados de SQL resuelvan un crimen intrigante.*

Ha ocurrido un crimen y el detective necesita tu ayuda. El detective te dio el informe de la escena del crimen, pero de alguna manera lo perdiste. Recuerda vagamente que **el crimen fue un asesinato que ocurrió en algún momento del 15 de enero de 2018** y que **tuvo lugar en SQL City**. Comience recuperando el informe de la escena del crimen correspondiente de la base de datos del departamento de policía.

<img src=https://raw.githubusercontent.com/NUKnightLab/sql-mysteries/master/schema.png>

In [1]:
# Usaremos la biblioteca sqlite3 de python para resolver el misterio

import pandas as pd
import sqlite3 as sql

In [2]:
# Establecer una conexión a la base de datos

conexion = sql.connect('sql-murder-mystery.db')

In [16]:
# Ejecutando nuestra primera consulta, buscando datos del informe de la escena del crimen
consulta_1 = '''
SELECT *
FROM crime_scene_report
WHERE city = "SQL City"
ORDER BY date;
'''

# Establecer el ancho del marco de datos al máximo
pd.set_option('display.max_colwidth', None)

# Ejecutando nuestra consulta
pd.read_sql_query(consulta_1, conexion)

,date,type,description,city
0,20170712,theft,"A lone hunter stalks the night, firing arrows into the Darkness.\n There is no hiding, no escape. In the distance, the beast\n falters, tethered to the void. The killing blow comes without\n hesitation, without mercy.",SQL City
1,20170820,arson,"Wield the Hammer of Sol with honor, Titan, it is a thing of\n legend, both past and future.",SQL City
2,20171110,robbery,"The Gjallarhorn shoulder-mounted rocket system was forged from\n the armor of Guardians who fell at the Twilight Gap. Gifted\n to the survivors of that terrible battle, the Gjallarhorn\n is seen as a symbol of honor and survival.",SQL City
3,20180103,bribery,"Apparently, Cayde thought it necessary to expose this extremely\n rare vegetable to a Hive summoning ritual.",SQL City
4,20180115,assault,"Hamilton: Lee, do you yield? Burr: You shot him in the side! Yes he yields!",SQL City
5,20180115,assault,Report Not Found,SQL City
6,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City
7,20180215,murder,REDACTED REDACTED REDACTED,SQL City
8,20180215,murder,Someone killed the guard! He took an arrow to the knee!,SQL City


Las imágenes de seguridad muestran que había 2 testigos. 

 - El primer testigo vive en la última casa de "Northwestern Dr". 

- El segundo testigo, llamado Annabel, vive en algún lugar de "Franklin Ave".

Tengo nuestro informe de la escena del crimen, según el informe hay 2 testigos. Vamos a buscarlos.

In [17]:
# Comprobación de los datos personales de ambos testigos
consulta_2 = '''
SELECT *
FROM person
WHERE address_street_name = "Northwestern Dr"
ORDER BY address_number desc;
'''
pd.read_sql_query(consulta_2, conexion)[:1] # Como buscamos en orden descendente, la primera tupla sera nuestro testigo

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [18]:
consulta_3 = '''
SELECT *
FROM person
WHERE name like '%Annabel%' AND address_street_name = "Franklin Ave";
'''
pd.read_sql_query(consulta_3, conexion)

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [19]:
# Vamos a ver la entrevista de ambos testigos tomada después del asesinato.
consulta_4 = '''
SELECT *
FROM interview
WHERE person_id = 14887 OR person_id = 16371;
'''
pd.read_sql_query(consulta_4, conexion)

,person_id,transcript
0,14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
1,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


Entrevistas:

- Morty Schapiro: Escuché un disparo y luego **vi a un hombre salir corriendo**. Tenía una **bolsa de "Get Fit Now Gym"**. El número de membresía en la bolsa comenzaba con **"48Z"**. **Solo los miembros oro tienen esas bolsas**. El hombre subió a un **automóvil con una placa que incluía "H42W"**.


- Annabel Miller: Vi el asesinato y **reconocí al asesino de mi gimnasio cuando estaba entrenando la semana pasada, el 9 de enero**.

Entonces, tenemos 2 pistas:

- Killer es un hombre y un miembro del gimnasio con un estatus de oro y con un número de membresía. comenzando con 48Z y se fue en un automóvil con un nro. de placa de H42W.

- Estaba haciendo ejercicio en el gimnasio el 9 de enero.

In [24]:
# Buscar el formato de fecha
consulta_fecha = '''
SELECT *
FROM get_fit_now_check_in
'''
pd.read_sql_query(consulta_fecha, conexion) #20180109

,membership_id,check_in_date,check_in_time,check_out_time
0,NL318,20180212,329,365
1,NL318,20170811,469,920
2,NL318,20180429,506,554
3,NL318,20180128,124,759
4,NL318,20171027,418,1019
...,...,...,...,...
2698,4KB72,20170422,1016,1114
2699,4KB72,20170630,408,885
2700,48Z7A,20180109,1600,1730
2701,48Z55,20180109,1530,1700


In [28]:
# Comprobando la base de datos del gimnasio con los detalles anteriores
consulta_5 = '''
SELECT *
FROM get_fit_now_check_in 
WHERE membership_id like "48Z%" AND check_in_date = 20180109 
order by check_in_date;
'''
pd.read_sql_query(consulta_5, conexion)

,membership_id,check_in_date,check_in_time,check_out_time
0,48Z7A,20180109,1600,1730
1,48Z55,20180109,1530,1700


Dos miembros encontrados y su identificación de membresía

In [25]:
# Ahora, verifiquemos los detalles del automóvil con los detalles anteriores
consulta_6 = '''
SELECT *
FROM drivers_license
WHERE plate_number like "%H42W%";
'''
pd.read_sql_query(consulta_6, conexion)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,183779,21,65,blue,blonde,female,H42W0X,Toyota,Prius
1,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
2,664760,21,71,black,black,male,4H42WR,Nissan,Altima


Dos hombres con una placa que contiene H42W

In [29]:
# Comprobando los datos personales de los dos hombres de la consulta anterior
consulta_7 = '''
SELECT *
FROM person
WHERE license_id = "423327" OR license_id = "664760";
'''
pd.read_sql_query(consulta_7, conexion)

,id,name,license_id,address_number,address_street_name,ssn
0,51739,Tushar Chandra,664760,312,Phi St,137882671
1,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


In [30]:
# Vamos a comprobar cuál de estos dos son miembros del gimnasio
consulta_8 = '''
SELECT *
FROM get_fit_now_member
WHERE person_id = "51739" OR person_id = "67318";
'''
pd.read_sql_query(consulta_8, conexion)

,id,person_id,name,membership_start_date,membership_status
0,48Z55,67318,Jeremy Bowers,20160101,gold


**Finalmente, encontramos al asesino - Jeremy Bowers.** 

Tanto la identificación como el estado de la membresía también coinciden con la información que encontramos anteriormente.

# Parte 2

¡Felicidades, encontraste al asesino! 

Pero espere, hay más... Si cree que está preparado para un desafío, **intente consultar la transcripción de la entrevista del asesino para encontrar al verdadero villano detrás de este crimen.**

In [31]:
# Hay más en esto, leyendo la transcripción del asesino
consulta_9 = '''
SELECT *
FROM interview
WHERE person_id = 67318;
'''
pd.read_sql_query(consulta_9, conexion)

,person_id,transcript
0,67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n"


Me contrató una mujer con mucho dinero. No sé su nombre, pero sé que mide alrededor de 5'5" (65") o 5'7" (67"). Es pelirroja y conduce un Tesla Model S. Sé que asistió al Concierto Sinfónico de SQL 3 veces en diciembre de 2017.


Entonces, el verdadero villano **es una mujer con un auto Tesla y cabello rojo**. Usando las pistas anteriores, averigüemos quién es el autor intelectual de este asesinato.

In [32]:
consulta_10 = '''
SELECT *
FROM drivers_license
WHERE car_make = "Tesla" AND car_model = "Model S" AND 
gender = "female" AND hair_color = "red";
'''
pd.read_sql_query(consulta_10, conexion)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,202298,68,66,green,red,female,500123,Tesla,Model S
1,291182,65,66,blue,red,female,08CM64,Tesla,Model S
2,918773,48,65,black,red,female,917UU3,Tesla,Model S


Tres mujeres con un Tesla Model S y cabello rojo

In [52]:
# Los datos personales de las tres mujeres anteriores son:
consulta_11 = '''
SELECT *
FROM person
WHERE license_id = "202298" OR license_id = "291182" OR license_id = "918773";
'''
pd.read_sql_query(consulta_11, conexion)

,id,name,license_id,address_number,address_street_name,ssn
0,78881,Red Korb,918773,107,Camerata Dr,961388910
1,90700,Regina George,291182,332,Maple Ave,337169072
2,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


In [56]:
# Verificando el evento SQL symphony concert
consulta_12 = '''
SELECT person_id, count(*) AS Asistencias_Concierto, event_name
FROM facebook_event_checkin
GROUP BY person_id
having count(*) = 3 AND event_name = "SQL Symphony Concert" AND date like "%201712%";
'''
pd.read_sql_query(consulta_12, conexion)

,person_id,Asistencias_Concierto,event_name
0,24556,3,SQL Symphony Concert
1,99716,3,SQL Symphony Concert


In [57]:
consulta_13 = '''
SELECT *, count(*) AS Asistencias_Concierto
FROM person
INNER JOIN facebook_event_checkin ON person.id=facebook_event_checkin.person_id
WHERE license_id = "202298" OR license_id = "291182" OR license_id = "918773"
GROUP BY person_id
having count(*) = 3 AND event_name = "SQL Symphony Concert" AND date like "%201712%";
'''
pd.read_sql_query(consulta_13, conexion)

,id,name,license_id,address_number,address_street_name,ssn,person_id,event_id,event_name,date,Asistencias_Concierto
0,99716,Miranda Priestly,202298,1883,Golden Ave,987756388,99716,1143,SQL Symphony Concert,20171206,3


**¡Felicitaciones, encontraste al cerebro detrás del asesinato!**

Todo el mundo en SQL City te aclama como el mejor detective de SQL de todos los tiempos. ¡Hora de romper el champán!